# Utilizando el servidor de nombres

El servidor de nombres permite que un cliente se contacte con un serivor de manera más sencilla.

Cuándo no se utiliza un servidor de nombres, se necesita especificar la IP y puerto del objeto remoto
en cada llamada.

In [2]:
from IPython.display import IFrame
IFrame(
    'https://pythonhosted.org/Pyro4/servercode.html#intermission-example-2-server-and-client-with-name-server',
    width='100%', height='500px')

# Arquitectura del NS
<img src="static/ns.svg?foo">

# Algunas consideraciones
Esta documento es un Notebook de Jupyter, cuyo kernel es un solo proceso.

Para poder lanzar procesos utilizaremos:

* Clase [`subporcess.Popen(comando)`](https://pythonhosted.org/Pyro4/servercode.html#intermission-example-2-server-and-client-with-name-server) para lanzar comandos en segundo plano. Lo utilizaremos para el comando `pyro4-ns`.
* Clase [`multiprocess.Process(target, args, kwargs)`](https://docs.python.org/2/library/multiprocessing.html) Permite lanzar un nuevo intérprete a a partir del estado actual, comenzando a ejecutar en la función `target`.
* Métodos `start()` y `stop()` en `Process`.
* Comando [`lsof`](http://linux.die.net/man/8/lsof) para detectar que procesos están escuchando en un puerto. En particular, el puerto del servidor de nombres.

In [3]:
from utils import lanzar_backgounrd
ok, why = lanzar_backgounrd('pyro4-ns', a_menos_que='lsof -t -i TCP:9090 -sTCP:LISTEN')
if ok:
    print("Se lanzó")
else:
    print("Ya estaba en ejecución")
    

Lanzando pyro4-ns con pid 26851
Se lanzó


### *El código de arriba es el equivalente a lanzarlo desde la consola:*
![Servidor de Nombres de Pyro4](static/pyro-ns.png)

Si lo lanzamos desde la consola, el servidor de nombres deberá estar en ejecución duranto todo el proceo de desarollo y eventual puesta en producción. No se activa como servicio del sistema por defecto, a diferencia de  [`rpcbind`](https://en.wikipedia.org/wiki/Portmap).

# Creando el servidor

In [8]:
import os
from Pyro4 import Daemon
from socket import gethostname

hostname = gethostname()

class Saludador(object):
    """
    Objeto a ser expuesto.
    """
    def saludar(self, nombre):
        """
        Retorna un saludo
        """
        return "Hola %s" % nombre
    
def exponer_objetos():
    # Crear una instancia
    saludador = Saludador()
    # serverSimple activa el loop, en este caso el objeto
    # se expone bajo el nombre saludador.
    Daemon.serveSimple(
    {
        saludador: "saludador"
    },
    ns=True, verbose=False, host=hostname)


# Ahora el cliente

In [9]:
from Pyro4 import Proxy

## Ahora el serivdor de nombres se encarga de mantener las ubicaciones

In [10]:
from utils import lanzar_como_proceso

with lanzar_como_proceso(exponer_objetos, warm_up_time=0.5):
    print("Primero le preguntamos al NS que objetos están disponibles:")
    !pyro4-nsc list
    objeto = Proxy('PYRONAME:saludador')
    print(objeto.saludar("Pepe"))

Lanzando <function exponer_objetos at 0x10a5c2f50> con el pid 26949.
Primero le preguntamos al NS que objetos están disponibles:
--------START LIST 
Pyro.NameServer --> PYRO:Pyro.NameServer@localhost:9090
saludador --> PYRO:obj_10cca021058f42ee9c7baf3b23e45bcd@greybird:53373
--------END LIST 
Hola Pepe


![Success](static/meme_success.jpg)